In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import re

kategori_list = ["nasional", "bisnis", "tekno", "bola", "metro", "dunia"]
base_rss = "https://rss.tempo.co"

results = []

for kategori in kategori_list:
    rss_url = f"{base_rss}/{kategori}"
    print(f"🔎 Ambil RSS: {rss_url}")
    resp = requests.get(rss_url, headers={"User-Agent": "Mozilla/5.0"})
    if resp.status_code != 200:
        print(f"❌ Gagal akses {rss_url}")
        continue

    soup = BeautifulSoup(resp.content, "xml")
    items = soup.find_all("item")

    for item in items:
        link = item.find("link").text
        judul = item.find("title").text.strip()

        try:
            berita_id = link.split("/")[-1].split("-")[0]
        except:
            berita_id = None

        # Ambil isi berita detail
        r = requests.get(link, headers={"User-Agent": "Mozilla/5.0"})
        if r.status_code != 200:
            continue
        s = BeautifulSoup(r.text, "html.parser")

        isi_tag = s.find("div", class_="detail-konten")
        if isi_tag:
            for tag in isi_tag(["script", "style", "aside", "figure"]):
                tag.decompose()
            isi = isi_tag.get_text(" ", strip=True)
            isi = re.sub(r"Baca juga.*?(?=[A-Z])", "", isi)
        else:
            isi = None

        results.append([berita_id, judul, isi, kategori])
        print(f"✅ {judul} [{kategori}]")

        time.sleep(1)

# Simpan ke CSV
with open("berita_tempo.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["id_berita", "judul", "isi", "kategori"])
    writer.writerows(results)

print(f"🎉 Crawling selesai, total berita: {len(results)}, hasil disimpan ke berita_tempo.csv")


🔎 Ambil RSS: https://rss.tempo.co/nasional
✅ Pertamina Hadirkan Teknologi Elektrolisis Modern untuk Produksi Green Hydrogen di Ulubelu [nasional]
✅ Banjir Landa Bali, Pemerintah Kota Denpasar Belum Keluarkan Status Kebencanaan [nasional]
✅ Aliansi Perempuan:  September Hitam Jangan Jadi Pengulangan [nasional]
✅ MUI Minta Program MBG Dihentikan Jika Ompreng Terbukti Tak Halal [nasional]
✅ Catatan Koalisi Sipil untuk DPR yang Akan Membahas RUU Perampasan Aset [nasional]
✅ MK Jadwalkan Sidang Pembuktian Lebih Lanjut Sengketa Pilkada Papua dan Barito Utara [nasional]
✅ Komisi X DPR Berharap Prabowo Cepat Umumkan Menpora Baru [nasional]
✅ Politikus PDIP Minta TNI Jelaskan Alasan Ferry Irwandi  Dianggap Ancaman Pertahanan Siber [nasional]
✅ Soal TNI Laporkan Ferry Irwandi, Sekjen Demokrat: Kalau Tidak Ada Indikasi Jangan Dibuat-buat [nasional]
✅ Kemensos Latih Ratusan Wali Asuh dan Wali Asrama Sekolah Rakyat [nasional]
✅ Wapres Gibran Tinjau Makan Bergizi Gratis di SMKN 1 Batam [nasional]
✅ 